In [61]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [62]:
filename = "ABox.ttl"

In [63]:
text1 = '''CQ_2.1
What are the textual descriptions assigned to the risks, the agents of deterioration classifying them, and their types?
'''

query1 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/02/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/02/data/>

SELECT ?risk ?agent_of_deterioration ?risk_type ?risk_summary
WHERE {
    ?risk_assessment a tbox:IdentificationDescription ;
                    tbox:identifies ?risk .
    ?risk a tbox:Risk ;
        tbox:isClassifiedByAgent ?agent_of_deterioration ;
        tbox:isClassifiedByType ?risk_type .
    OPTIONAL {
        ?risk_assessment tbox:hasNote ?risk_summary .
    }
}
'''

In [64]:
text2 = '''CQ_2.2
Which risks are identified within the layers of the site or region, along with their types, the documents documenting them, and the start and end dates of the time intervals in which they have been identified?
'''

query2 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/02/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/02/data/>

SELECT ?risk ?risk_type ?document ?start_date ?end_date
WHERE {
    ?risk_assessment a tbox:IdentificationDescription ;
                    tbox:isDocumentedBy ?document ;
                    tbox:atTime ?time_interval ;
                    tbox:identifies ?risk .
    ?time_interval tbox:hasStartDate ?start_date ;
                    tbox:hasEndDate ?end_date .
    ?risk a tbox:Risk ;
            tbox:isClassifiedByType ?risk_type ;
            tbox:isClassifiedByLayer ?layer .
    FILTER (
        ?layer = tbox:site ||
        ?layer = tbox:region
    )
}
'''

In [65]:
queries = [(text1, query1),
           (text2, query2)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_2.1
Return the textual descriptions assigned to the risks, the agents of deterioration that classify them, and their types.

+------------------------------------------------------------------------+-----------------------------------------------------------------------+------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
| risk                                                                   | agent_of_deterioration                                                | risk_type                                                        | risk_summary                                                                                                                                                  |
|------------------------------------------------------------------------+--------------------------------------------------